In [8]:
from llama_index import SimpleDirectoryReader, LangchainEmbedding, GPTListIndex,GPTSimpleVectorIndex, PromptHelper
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LLMPredictor, ServiceContext
import torch
from langchain.llms.base import LLM
from transformers import pipeline

In [9]:
class customLLM(LLM):
    model_name = "google/flan-t5-large"
    pipeline = pipeline("text2text-generation", model=model_name, device="cpu", model_kwargs={"torch_dtype":torch.bfloat16})

    def _call(self, prompt, stop=None):
        return self.pipeline(prompt, max_length=9999)[0]["generated_text"]
 
    def _identifying_params(self):
        return {"name_of_model": self.model_name}

    def _llm_type(self):
        return "custom"


llm_predictor = LLMPredictor(llm=customLLM())

In [10]:
hfemb = HuggingFaceEmbeddings()
embed_model = LangchainEmbedding(hfemb)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu


In [11]:
from llama_index import Document

with open('./output/sample_health_insurance_policy.txt', 'r') as f:
    text = f.read()

text_list = [text]

documents = [Document(t) for t in text_list]

In [12]:
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, embed_model=embed_model)
index = GPTSimpleVectorIndex.from_documents(documents, service_context=service_context)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 5901 tokens


In [13]:
import logging

logging.getLogger().setLevel(logging.CRITICAL)

In [17]:
query = 'What is the overall deductible?'
query = 'What is the copay for Diagnostic test?'
response = index.query(query)

In [18]:
response

Response(response='10 of 8 Insurance Company 1: Plan Option 1 Coverage Period: 01/01/2013 – 12/31/2013 Summary of Benefits and Coverage: What this Plan Covers & What it Costs Coverage for: Individual + Spouse | Plan Type: PPO ** Individual health insurance sample – ** Group health coverage sample – Your Rights to Continue Coverage: Federal and State laws may provide protections that allow you to keep this health insurance coverage as long as you pay your premium. There are exceptions, however, such as if: You commit fraud The insurer stops offering services in the State You move outside the coverage area For more information on your rights to continue coverage, contact the insurer at [contact number]. You may also contact your state insurance department at [insert applicable State Department of Insurance contact information]. OR Your Grievance and Appeals Rights: If you lose coverage under the plan, then, depending upon the circumstances, Federal and State laws may provide protections 

In [16]:
response

Response(response='$500 person / $1,000 family', source_nodes=[NodeWithScore(node=Node(text='Insurance Company 1: Plan Option 1 \nCoverage Period: 01/01/2013 – 12/31/2013 \nSummary of Benefits and Coverage: What this Plan Covers & What it Costs  Coverage for: Individual + Spouse | Plan Type: PPO \n\nThis is only a summary. If you want more detail about your coverage and costs, you can get the complete terms in the policy or plan \ndocument at www.[insert] or by calling 1-800-[insert]. \n\n \n\n \n\n \n\nImportant Questions  Answers \n\nWhy this Matters: \n\nWhat is the overall \ndeductible? \n\n$500 person /  \n$1,000 family  \nDoesn’t apply to preventive care  \n\nYou must pay all the costs up to the deductible amount before this plan begins to pay for \ncovered services you use. Check your policy or plan document to see when the deductible \nstarts over (usually, but not always, January 1st). See the chart starting on page 2 for how \nmuch you pay for covered services after you meet 

In [15]:
from IPython.display import display
import ipywidgets as widgets

In [16]:
chat_history = []

def on_submit(_):
    query = input_box.value
    input_box.value = ""
    
    if query.lower() == 'exit':
        print("Thank you for using the State of the Union chatbot!")
        return
    
    result = index.query(query)
    chat_history.append((query, result['answer']))
    
    display(widgets.HTML(f'<b>User:</b> {query}'))
    display(widgets.HTML(f'<b><font color="blue">Chatbot:</font></b> {result["answer"]}'))

print("Welcome to the Transformers chatbot! Type 'exit' to stop.")

input_box = widgets.Text(placeholder='Please enter your question:')
input_box.continuous_update = False
input_box.observe(on_submit, names='value')
# input_box.on_submit(on_submit)

display(input_box)

Welcome to the Transformers chatbot! Type 'exit' to stop.


Text(value='', continuous_update=False, placeholder='Please enter your question:')